This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [1]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

In [3]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2018-06-27&end_date=2018-06-27&apikey='+API_KEY)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(r.json())

{'dataset': {'id': 10095370, 'dataset_code': 'AFX_X', 'database_code': 'FSE', 'name': 'Carl Zeiss Meditec (AFX_X)', 'description': 'Stock Prices for Carl Zeiss Meditec (AFX) from the Frankfurt Stock Exchange.<br><br>Trading System: Xetra<br><br>ISIN: DE0005313704', 'refreshed_at': '2018-08-02T22:31:15.721Z', 'newest_available_date': '2018-08-02', 'oldest_available_date': '2000-06-07', 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'frequency': 'daily', 'type': 'Time Series', 'premium': False, 'limit': None, 'transform': None, 'column_index': None, 'start_date': '2018-06-27', 'end_date': '2018-06-27', 'data': [['2018-06-27', None, 59.65, 58.6, 58.95, None, 96154.0, 5685432.0, None, None, None]], 'collapse': None, 'order': None, 'database_id': 6129}}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

Task 1) Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).

In [5]:
r = requests.get('https://www.quandl.com/api/v3/datasets/FSE/AFX_X?start_date=2017-01-01&end_date=2017-12-31&apikey='+API_KEY)

Task 2) Convert the returned JSON object into a Python dictionary.

In [6]:
afx_2017_dict = r.json()

In [7]:
type(afx_2017_dict)

dict

Task 3) Calculate what the highest and lowest opening prices were for the stock in this period.

In [8]:
data = afx_2017_dict['dataset']['data']
opening = []
for i in data:
    if i[1] != None:
        opening.append(i[1])
    
max_open = max(opening)
min_open = min(opening)

print('Max opening price in 2017: ', max_open)
print('Min opening price in 2017: ', min_open)

Max opening price in 2017:  53.11
Min opening price in 2017:  34.0


Task 4) What was the largest change in any one day (based on High and Low price)?

In [36]:
change = []

for i in data:
    change.append(abs(i[2] - i[3]))

max_change = round(max(change), 2)

print('Max daily price change in 2017: ', max_change)    

Max daily price change in 2017:  2.81


Task 5) What was the largest change between any two days (based on Closing Price)?

In [9]:
dod_change = [] 

for i in range(len(data) - 1):
    dod_change.append(abs(data[i][4] - data[i+1][4]))
        
largest_change = round(max(dod_change),2)

print('Largest day-over-day change: ', largest_change)

Largest day-over-day change:  2.56


Task 6) What was the average daily trading volume during this year?

In [10]:
trade_volume = []
for i in data:
    if i[6] != None:
        trade_volume.append(int(i[6]))

avg_trade_volume = sum(trade_volume) / len(trade_volume)

print('Average daily trading volume in 2017: ', avg_trade_volume)  

Average daily trading volume in 2017:  89124.33725490196


Task 7) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [12]:
def calc_median(int_list):
    list_sorted = sorted(int_list)
    length = len(int_list)
    if length % 2 == 1:
        return list_sorted[int((length-1)/2)]
    else:
        index1 = int(length / 2)
        index2 = int(index1 - 1)
        return (list_sorted[index1] + list_sorted[index2]) / 2

median_volume = calc_median(trade_volume)

print('Median trade volume in 2017: ', median_volume) # s/b 76,286


Median trade volume in 2017:  76286
